In [57]:
import sys
sys.path.append("..")

## Set Seed

In [58]:
import torch
import numpy as np
import random
import os

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)

  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

set_seed(42)

In [59]:
from datasets import load_dataset
import numpy as np

# Load the dataset
dataset = load_dataset("jtz18/skin-lesion")

c:\Users\jonta\code\school\t8\DeepLearning\50.039-DL\venv\lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for jtz18/skin-lesion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/jtz18/skin-lesion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [60]:
import torch

class OneClassDataset(torch.utils.data.Dataset):
    def __init__(self, dataset=None, split="train", transform=None):
        self.dataset = dataset
        self.split = split
        self.transform = transform

    def __len__(self):
        return len(self.dataset[self.split])

    def __getitem__(self, idx):
        sample = self.dataset[self.split][idx]
        image = sample['image']
        mask = sample['label']
        image = np.array(image.convert('RGB'))
        mask = np.array(mask.convert('L'), dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations['image']
            mask = augmentations['mask']

        return image, mask

In [61]:
import os
import torch
import torchvision
from torch.utils.data import DataLoader
import wandb

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    # Create the 'checkpoints' directory if it doesn't exist
    if not os.path.isdir("checkpoints"):
        os.makedirs("checkpoints")

    # Save the checkpoint to the 'checkpoints' directory
    torch.save(state, os.path.join("checkpoints", filename))

def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    epoch = checkpoint["epoch"]
    return model, optimizer, epoch

def get_loaders(
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_ds = OneClassDataset(
        dataset=dataset,
        split="train",
        transform=train_transform,
    )

    # # Use only the first 100 samples of the dataset
    # train_ds = torch.utils.data.Subset(train_ds, range(100))

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = OneClassDataset(
        dataset=dataset,
        split="validation",
        transform=val_transform,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    test_ds = OneClassDataset(
        dataset=dataset,
        split="test",
        transform=val_transform,
    )

    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader, test_loader

def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    iou_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )
            iou_score += iou_pytorch(preds, y)  # Calculate IoU score for the batch

    pixel_accuracy = num_correct / num_pixels * 100
    print(f"dice_score: {dice_score}")
    dice_score = dice_score / len(loader)
    print(f"Got {num_correct}/{num_pixels} with acc {pixel_accuracy:.2f}")
    wandb.log({"Pixel Accuracy": pixel_accuracy})
    print(f"Dice score: {dice_score}")
    wandb.log({"Dice Score": dice_score})
    print(f"IoU score: {iou_score}")  # Print IoU score to the console
    wandb.log({"IoU Score": iou_score})
    model.train()

def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
    # Convert tensors to 'Bool' type
    outputs_int = outputs.int()
    labels_int = labels.int()

    intersection = (outputs_int & labels_int).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    union = (outputs_int | labels_int).float().sum((1, 2))  # Will be zero if both are 0

    iou = (intersection + 1e-6) / (union + 1e-6)  # We smooth our division to avoid 0/0

    return iou.mean()  # Average over the batch


def save_predictions_as_imgs(
    loader, model, folder="saved_images", device="cuda"
):

    # Create directory if it doesn't exist
    if not os.path.exists(folder):
        os.makedirs(folder)

    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            # print(f"pred_shape: {preds.shape}")
            # print(f"y_shape: {y.shape}")

        torchvision.utils.save_image(
            preds, f"{folder}/pred_{idx}.png"
        )
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}/{idx}.png")

    model.train()

## Init Wandb

In [62]:
# Hyperparameters etc.
import wandb
from src.segFormer import Segformer

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

LEARNING_RATE = 1e-5

DEVICE = device
BATCH_SIZE = 16

NUM_EPOCHS = 50

NUM_WORKERS = 0

IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256

PIN_MEMORY = True
LOAD_MODEL = False
CHECKPOINT_FILENAME = ""   # Model checkpoint filename if LOAD is True eg: checkpoints/checkpoint_9.pth.tar or None
CLASS = "task1"
model = Segformer().to(DEVICE)
MODEL = model.__class__.__name__

# Initialize a new run
run = wandb.init(project="unet-skin-lesion", config={
    "learning_rate": LEARNING_RATE,
    "device": DEVICE,
    "batch_size": BATCH_SIZE,
    "num_epochs": NUM_EPOCHS,
    "num_workers": NUM_WORKERS,
    "image_height": IMAGE_HEIGHT,
    "image_width": IMAGE_WIDTH,
    "pin_memory": PIN_MEMORY,
    "load_model": LOAD_MODEL,
    "checkpoint_filename": CHECKPOINT_FILENAME,
    "class": CLASS,
})

Dice Score,▁▁
IoU Score,▁▁
Pixel Accuracy,▁▁
Training Loss,▆▅▃▅▄▃▃▆▂▃▄▁▁█
Dice Score,0.0
IoU Score,3.41309
Pixel Accuracy,73.16641
Training Loss,0.69055


In [63]:
from tqdm import tqdm
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)
            wandb.log({"Training Loss": loss})

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

In [64]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch import nn


train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

val_transforms = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


train_loader, val_loader, test_loader = get_loaders(
    BATCH_SIZE,
    train_transform,
    val_transforms,
    NUM_WORKERS,
    PIN_MEMORY,
)

In [65]:
start_epoch = 0
if LOAD_MODEL:
    checkpoint = torch.load(CHECKPOINT_FILENAME)
    model, optimizer, start_epoch = load_checkpoint(checkpoint, model, optimizer)



check_accuracy(val_loader, model, device=DEVICE)
scaler = torch.cuda.amp.GradScaler()

for epoch in range(start_epoch, NUM_EPOCHS):
    print(f"epoch: {epoch}")
    train_fn(train_loader, model, optimizer, loss_fn, scaler)

    # save model
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer":optimizer.state_dict(),
        "epoch": epoch,
    }
    save_checkpoint(checkpoint, filename=f"checkpoint_{epoch}.pth.tar")

    # check accuracy
    check_accuracy(val_loader, model, device=DEVICE)

    # print some examples to a folder
    save_predictions_as_imgs(
        val_loader, model, folder=f"saved_images/{epoch}", device=DEVICE
    )

dice_score: 0.0
Got 4795034/6553600 with acc 73.17
Dice score: 0.0
IoU score: 3.4130859375
epoch: 0


100%|██████████| 163/163 [06:28<00:00,  2.38s/it, loss=0.786]


=> Saving checkpoint
dice_score: 3.109121322631836
Got 5192275/6553600 with acc 79.23
Dice score: 0.44416022300720215
IoU score: 4.271749496459961
epoch: 1


100%|██████████| 163/163 [06:23<00:00,  2.35s/it, loss=0.265]


=> Saving checkpoint
dice_score: 5.33040714263916
Got 5797344/6553600 with acc 88.46
Dice score: 0.7614867687225342
IoU score: 5.313266754150391
epoch: 2


...

=> Saving checkpoint
dice_score: 6.116086006164551
Got 6094130/6553600 with acc 92.99
Dice score: 0.8737266063690186
IoU score: 5.924324035644531
epoch: 48


100%|██████████| 163/163 [06:30<00:00,  2.40s/it, loss=0.0167]


=> Saving checkpoint
dice_score: 6.0824737548828125
Got 6073719/6553600 with acc 92.68
Dice score: 0.8689248561859131
IoU score: 5.930449485778809
epoch: 49


100%|██████████| 163/163 [06:28<00:00,  2.38s/it, loss=0.151] 


=> Saving checkpoint
dice_score: 5.861730098724365
Got 5998501/6553600 with acc 91.53
Dice score: 0.8373900651931763
IoU score: 5.8015618324279785


In [66]:
run.finish()

Dice Score,▁▅▇▇▇▇▇███▇██▇██████████████████████████
IoU Score,▁▃▆▇▆▇▇▇▇▇▇█▇▇█▇▇▇████▇▇█▇▇██▇██████████
Pixel Accuracy,▁▃▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇█▇██▇▇██▇██▇█████████▇
Training Loss,█▅▂▄▂▃▃▂▅▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▅▂▂▂▂▁▂▃▂▂▄▂▂▂▃▂
Dice Score,0.83739
IoU Score,5.80156
Pixel Accuracy,91.52986
Training Loss,0.15081
